In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML

This Python function, inverse_scanner, is implementing an inverse range sensor model for a robotic system (like an autonomous vehicle) that's equipped with a laser rangefinder or a LiDAR. It's used to transform raw sensor measurements into a form suitable for updating a probabilistic occupancy grid.

Here's a line-by-line explanation of the code:

The function definition. It takes several arguments:

num_rows, num_cols are the dimensions of the map or grid.
x, y, theta are the position and orientation of the robot.
meas_phi, meas_r are arrays representing the angular positions and ranges of the laser measurements.
rmax is the maximum range of the sensor.
alpha and beta are parameters of the sensor model related to the width of the laser beam and the field of view of the sensor, respectively.
It initializes an empty occupancy grid m of size MxN with zeros. M and N should presumably be num_rows and num_cols.

The outer two for loops iterate over each cell in the grid.

For each cell, it calculates r, the distance from the robot to the cell, and phi, the bearing (angle) from the robot to the cell, relative to the robot's heading.

It identifies the laser beam k whose angular position is closest to phi.

If the cell is outside the laser's maximum range, or behind the range measurement of the identified laser beam, or outside the beam's field of view, then the occupancy of the cell is unknown. It's marked with a probability of 0.5.

If the laser beam k hit an object (its range measurement is less than the maximum range), and the cell is located at a distance close to the range measurement, then the cell is likely to be occupied by an object. It's marked with a higher probability of 0.7.

If the cell is in front of the range measurement of the laser beam, then it's likely to be free space, as the beam didn't hit anything at this distance. It's marked with a lower probability of 0.3.

After it checks all the cells, the function returns the updated occupancy grid.

The purpose of this function is to interpret the raw laser measurements and translate them into a probabilistic understanding of which areas in the robot's vicinity are occupied and which are likely to be free. This occupancy information can then be used to plan safe and efficient paths for the robot.

In [3]:
# Calculates the inverse measurement model for a laser scanner.
# It identifies three regions. The first where no information is available occurs
# outside of the scanning arc. The second where objects are likely to exist, at the
# end of the range measurement within the arc. The third are where objects are unlikely
# to exist, within the arc but with less distance than the range measurement.
def inverse_scanner(num_rows, num_cols, x, y, theta, meas_phi, meas_r, rmax, alpha, beta):
    m = np.zeros((M, N))
    for i in range(num_rows):
        for j in range(num_cols):
            # Find range and bearing relative to the input state (x, y, theta).
            r = math.sqrt((i - x)**2 + (j - y)**2)
            phi = (math.atan2(j - y, i - x) - theta + math.pi) % (2 * math.pi) - math.pi
            
            # Find the range measurement associated with the relative bearing.
            k = np.argmin(np.abs(np.subtract(phi, meas_phi)))
            
            # If the range is greater than the maximum sensor range, or behind our range
            # measurement, or is outside of the field of view of the sensor, then no
            # new information is available.
            if (r > min(rmax, meas_r[k] + alpha / 2.0)) or (abs(phi - meas_phi[k]) > beta / 2.0):
                m[i, j] = 0.5
            
            # If the range measurement lied within this cell, it is likely to be an object.
            elif (meas_r[k] < rmax) and (abs(r - meas_r[k]) < alpha / 2.0):
                m[i, j] = 0.7
            
            # If the cell is in front of the range measurement, it is likely to be empty.
            elif r < meas_r[k]:
                m[i, j] = 0.3
                
    return m
                            

In [4]:
# Generates range measurements for a laser scanner based on a map, vehicle position,
# and sensor parameters.
# Uses the ray tracing algorithm.
def get_ranges(true_map, X, meas_phi, rmax):
    (M, N) = np.shape(true_map)
    x = X[0]
    y = X[1]
    theta = X[2]
    meas_r = rmax * np.ones(meas_phi.shape)
    
    # Iterate for each measurement bearing.
    for i in range(len(meas_phi)):
        # Iterate over each unit step up to and including rmax.
        for r in range(1, rmax+1):
            # Determine the coordinates of the cell.
            xi = int(round(x + r * math.cos(theta + meas_phi[i])))
            yi = int(round(y + r * math.sin(theta + meas_phi[i])))
            
            # If not in the map, set measurement there and stop going further.
            if (xi <= 0 or xi >= M-1 or yi <= 0 or yi >= N-1):
                meas_r[i] = r
                break
            # If in the map, but hitting an obstacle, set the measurement range
            # and stop ray tracing.
            elif true_map[int(round(xi)), int(round(yi))] == 1:
                meas_r[i] = r
                break
                
    return meas_r